In [2]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [3]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [6]:
#URM_train, URM_val = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")

In [7]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions


In [8]:
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [35]:
def objective_function_KNN(optuna_trial):
                       
    similarity="tanimoto"
    
    recommender_instance = ItemKNNCFRecommender(URM_train)
    
    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 2, 50),
                   "shrink": optuna_trial.suggest_float("shrink", 1 , 25),
                   "feature_weighting": optuna_trial.suggest_categorical("feature_weighting", ["BM25", "TF-IDF", "none"]),
                   "normalize": optuna_trial.suggest_categorical("normalize", [True, False])
                  }
    
    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0.1, 0.99, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0.1, 1.9, log=False)
        full_hyperp["normalize"] = True 

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])
        
    
    recommender_instance.fit(**full_hyperp)
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)    
    return eval_res["MAP"][10]

In [36]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame()
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        
        self.results_df = self.results_df._append(hyperparam_dict, ignore_index=True)

In [37]:
optuna_study = op.create_study(direction="maximize")
        
save_results_tanimoto = SaveResults()
        
optuna_study.optimize(objective_function_KNN,
                      callbacks=[save_results_tanimoto],
                      n_trials = 300)

[I 2024-12-14 14:06:37,031] A new study created in memory with name: no-name-0ca75cd6-e003-467d-8664-ba50b41fb5be


Similarity column 38121 (100.0%), 5329.18 column/sec. Elapsed time 7.15 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 9.12 sec. Users per second: 3808


[I 2024-12-14 14:06:53,439] Trial 0 finished with value: 0.025410928192976807 and parameters: {'topK': 29, 'shrink': 13.098235730029419, 'feature_weighting': 'none', 'normalize': True}. Best is trial 0 with value: 0.025410928192976807.


Similarity column 38121 (100.0%), 5317.13 column/sec. Elapsed time 7.17 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 9.25 sec. Users per second: 3756


[I 2024-12-14 14:07:09,973] Trial 1 finished with value: 0.025133588056661143 and parameters: {'topK': 36, 'shrink': 7.461661363209599, 'feature_weighting': 'none', 'normalize': True}. Best is trial 0 with value: 0.025410928192976807.


...

[I 2024-12-14 15:40:35,836] Trial 297 finished with value: 0.026262357943637606 and parameters: {'topK': 7, 'shrink': 24.047359954756924, 'feature_weighting': 'TF-IDF', 'normalize': False}. Best is trial 231 with value: 0.026510516650028845.


Similarity column 38121 (100.0%), 5305.13 column/sec. Elapsed time 7.19 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.53 sec. Users per second: 4072


[I 2024-12-14 15:40:51,703] Trial 298 finished with value: 0.026510516650028845 and parameters: {'topK': 5, 'shrink': 24.33024928597077, 'feature_weighting': 'TF-IDF', 'normalize': False}. Best is trial 231 with value: 0.026510516650028845.


Similarity column 38121 (100.0%), 5162.90 column/sec. Elapsed time 7.38 sec
EvaluatorHoldout: Processed 34728 (100.0%) in 8.64 sec. Users per second: 4018


[I 2024-12-14 15:41:07,877] Trial 299 finished with value: 0.024445290018026297 and parameters: {'topK': 4, 'shrink': 23.568471338458384, 'feature_weighting': 'BM25', 'normalize': False}. Best is trial 231 with value: 0.026510516650028845.


In [39]:
save_results_tanimoto.results_df.to_csv("result_experiments/ItemKNNCFRecommender/results_tanimoto.csv")

In [18]:
save_results_cosine.results_df['result'].max() 

0.0263499559387598

In [33]:
save_results_adjusted.results_df.to_csv("result_experiments/ItemKNNCFRecommender/results_adjusted.csv")